In [19]:
import numpy as np, pandas as pd, json, glob, os, openai
# from sentence_transformers import SentenceTransformer
from openai import OpenAI
from typing import List, Tuple, Union
from langchain_ollama.embeddings import OllamaEmbeddings
from tqdm.auto import tqdm
# import shutil
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


In [3]:
client = QdrantClient(path="../data/qdrant_local")  # 데이터는 ./qdrant_local 폴더에 저장됨

# 2️⃣ 컬렉션 이름 및 파라미터 정의
collection_name = "medical_qa"

In [15]:
client.delete_collection(collection_name=collection_name)

True

In [5]:
# client.create_collection(
#         collection_name=collection_name,
#         vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
#     )
model = SentenceTransformer("BAAI/bge-m3")


In [11]:
path+i

'../../../medical_data/02.라벨링데이터/TL_내과'

In [ ]:
path = '../../../data/1.데이터/Training/02.라벨링데이터/'
path = '../../../medical_data/02.라벨링데이터/'
# os.listdir(path)
# for name_ in os.listdir(path):
#     print(name_)
#     collection_name = name_

# NOTE Create Collection
# client.recreate_collection(
# collection_name=collection_name,
# vectors_config=VectorParams(size=1024, distance=Distance.COSINE))
# for i in tqdm(os.listdir(path)):
#     for j in tqdm(os.listdir(path+'/'+i)):
#         with open(path+i+'/'+j, "r", encoding="utf-8-sig") as f:
#             data = json.load(f)
        
        
#         embedding = model.encode(data["question"]).tolist()
        
#         client.upsert(
#         collection_name=collection_name,
#         points=[
#             PointStruct(
#                 id=data["qa_id"],
#                 vector=embedding,
#                 payload={
#                     "domain": data["domain"],
#                     "q_type": data["q_type"],
#                     "question": data["question"],
#                     "answer": data["answer"],
#                     },
#                 )
#             ],
#         )
#     print(f"{i} done===============")
    
        
        
        
        
        
    #     break
    # break        

/tmp/ipykernel_1082/4247610530.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [11:37<34:53, 697.96s/it]

TL_내과 done===============


 50%|█████     | 2/4 [12:17<10:21, 310.57s/it]

TL_응급의학과 done===============


 75%|███████▌  | 3/4 [14:26<03:47, 227.85s/it]

TL_산부인과 done===============


100%|██████████| 4/4 [17:07<00:00, 256.87s/it]

TL_소아청소년과 done===============


# Question

In [20]:
query_data = {
    "qa_id": 4286,
    "domain": 17,
    "q_type": 1,
    "question": "65세 남자가 심질환으로 현재 aspirin을 복용 중이다. 이 환자에서 aspirin 복용으로 인해 나타날 수 있는 소견으로 가장 적절한 것은?  \n1) PTT 증가  \n2) TT 증가  \n3) 혈소판 감소  \n4) PT 증가  \n5) BT 증가",
    "answer": "5) BT 증가"
}

# domain, q_type 기준으로 필터 설정
filters = models.Filter(
    must=[
        models.FieldCondition(
            key="domain",
            match=models.MatchValue(value=query_data["domain"])
        ),
        models.FieldCondition(
            key="q_type",
            match=models.MatchValue(value=query_data["q_type"])
        ),
    ]
)

In [21]:
query_vector = model.encode(query_data["question"]).tolist()

In [22]:
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3,
    query_filter=filters
)

/tmp/ipykernel_1082/1981553409.py:1: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


In [25]:
for i, r in enumerate(results):
    print(f"{i+1}. Score: {r.score:.4f}")
    print(f"Question: {r.payload['question']}")
    print(f"Answer: {r.payload['answer']}\n")


1. Score: 0.7091
Question: 65세 남성이 급성 심근경색으로 응급실에 내원하였다. 이 환자의 초기 치료로 권장되는 aspirin의 최소 부하 용량은 얼마인가?  
1) 50 mg  
2) 75 mg  
3) 100 mg  
4) 160~325 mg  
5) 500 mg
Answer: 4) 160~325 mg

2. Score: 0.7024
Question: 65세 남성이 관상동맥 스텐트 시술 후 항혈전 치료를 받고 있다. 티클로피딘과 아스피린을 병용 투여할 경우, 다음 중 주요한 치료 효과로 기대되는 것은 무엇인가?  
1) 총 사망률 감소  
2) 혈전성 혈소판 감소성 자반증(TTP) 발생 감소  
3) 비치명적 심근경색 발생 감소  
4) 백혈구 감소증 발생 감소  
5) 경미한 출혈 증가
Answer: 3) 비치명적 심근경색 발생 감소

3. Score: 0.6967
Question: 60세 남성이 갑작스러운 심한 가슴 통증으로 응급실에 내원하였습니다. 심전도에서 ST 분절 상승이 관찰되었고, 혈액 검사에서 트로포닌 수치가 상승하였습니다. 이 환자에서 가장 우선적으로 시행해야 할 치료는 무엇인가요?

1) 심장 초음파  
2) 혈전 용해제 투여  
3) 응급 관상동맥 중재술(PCI) 
4) 니트로글리세린 설하정  
5) 아스피린 복용
Answer: 3) 응급 관상동맥 중재술(PCI)



In [ ]:
# 3️⃣ 임베딩 모델 로드 (bge-m3)
model = SentenceTransformer("BAAI/bge-m3")

# 5️⃣ question 임베딩 생성
# embedding = model.encode(data["question"]).tolist()

# 6️⃣ Qdrant에 데이터 업로드
client.upsert(
    collection_name=collection_name,
    points=[
        PointStruct(
            id=data["qa_id"],
            vector=embedding,
            payload={
                "domain": data["domain"],
                "q_type": data["q_type"],
                "question": data["question"],
                "answer": data["answer"],
            },
        )
    ],
)

print("✅ 데이터 업로드 완료!")

✅ 데이터 업로드 완료!


In [5]:
if collection_name not in [c.name for c in client.get_collections().collections]:
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=1024, distance=Distance.COSINE)
    )
    print(f"✅ 컬렉션 '{collection_name}' 생성 완료")
else:
    print(f"ℹ️ 컬렉션 '{collection_name}' 이미 존재")

ℹ️ 컬렉션 'medical_qa' 이미 존재
